# R-NaD Pokemon TCG Pocket Training on Colab

This notebook sets up the environment and runs the training script for R-NaD on Google Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!uv pip install --system  mlflow pyngrok -q

from google.colab import userdata
authtoken = userdata.get('ngrok_token')
!ngrok config add-authtoken {authtoken}

%env JAX_LOG_COMPILES=1

In [ ]:
# @title Clone Repository
import os

os.chdir("/content")

REPO_URL = "https://github.com/kitamura-tetsuo/R-NaD-PokemonTCGP.git"
REPO_NAME = "R-NaD-PokemonTCGP"

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_NAME}...")
    !git clone --recursive {REPO_URL}
else:
    print(f"{REPO_NAME} already exists. Pulling latest changes...")
    os.chdir(REPO_NAME)
    !git pull
    !git submodule update --init --recursive
    os.chdir("..")

os.chdir(REPO_NAME)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title Setup Environment
import os
import subprocess

# 1. Install Rust (required for deckgym-core)
print("Installing Rust...")
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
os.environ["PATH"] += ":/root/.cargo/bin"

# 2. Install uv
print("Installing uv...")
!curl -LsSf https://astral.sh/uv/install.sh | sh

# 3. Check TPU/GPU and install JAX
try:
    import jax
except ImportError:
    pass

if 'TPU_ACCELERATOR_TYPE' in os.environ:
    print("TPU detected via TPU_ACCELERATOR_TYPE. Installing JAX for TPU...")
    !uv pip install --system "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
else:
    print("TPU not detected. Installing JAX for CUDA...")
    !nvidia-smi
    !uv pip install --system "jax[cuda12]"

# 4. Install other dependencies
print("Installing other Python dependencies...")
!uv pip install --system dm-haiku optax open_spiel mlflow orbax-checkpoint maturin gymnasium

# 5. Build deckgym-core (Manual Cargo Build)
if os.path.exists("deckgym-core"):
    print("Building deckgym-core manually with cargo...")
    
    # 1. Setup Rust environment
    home = os.path.expanduser("~")
    cargo_bin = os.path.join(home, ".cargo", "bin")
    if cargo_bin not in os.environ["PATH"]:
        os.environ["PATH"] += f":{cargo_bin}"
    
    # 2. Build shared library directly using cargo
    # This bypasses maturin and pip entirely to avoid TPU runtime issues
    print("Running cargo build --release --features python...")
    build_cmd = f"export PATH={cargo_bin}:$PATH && cd deckgym-core && cargo build --release --features python"
    if os.system(build_cmd) != 0:
        raise RuntimeError("Cargo build failed")
    
    # 3. Locate and move the compiled library
    # cargo produces libdeckgym.so in target/release/
    # We need to move it to deckgym-core/python/deckgym/deckgym.so
    src_lib = "deckgym-core/target/release/libdeckgym.so"
    dst_lib = "deckgym-core/python/deckgym/deckgym.so"
    
    if os.path.exists(src_lib):
        print(f"Copying {src_lib} to {dst_lib}")
        !cp {src_lib} {dst_lib}
    else:
        raise FileNotFoundError(f"Compiled library not found at {src_lib}")
    
    # 4. Install the python package by copying it to the working directory
    # This relies on the current directory being in PYTHONPATH
    if os.path.exists("deckgym-core/python/deckgym"):
        print("Copying deckgym package to current directory...")
        !rm -rf deckgym
        !cp -r deckgym-core/python/deckgym .
    
    # 5. Copy deckgym_openspiel
    if os.path.exists("deckgym-core/python/deckgym_openspiel"):
        print("Copying deckgym_openspiel to current directory...")
        !rm -rf deckgym_openspiel
        !cp -r deckgym-core/python/deckgym_openspiel .
        
    # 6. Verify import
    try:
        import deckgym
        print("Successfully imported deckgym from:", deckgym.__file__)
    except ImportError as e:
        print(f"Failed to import deckgym: {e}")
else:
    print("Warning: deckgym-core not found.")

In [ ]:
# @title Run Training with Drive Output
import sys
import os

# Add paths to PYTHONPATH
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "deckgym-core/python"))
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.path.join(os.getcwd(), 'deckgym-core/python')}"

# Define Drive Paths
DRIVE_ROOT = "/content/drive/MyDrive/R-NaD-PokemonTCGP_experiments"
CHECKPOINT_DIR = os.path.join(DRIVE_ROOT, "checkpoints")
MLRUNS_DIR = os.path.join(DRIVE_ROOT, "mlruns")

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(MLRUNS_DIR, exist_ok=True)

# Set MLflow Tracking URI
os.environ["MLFLOW_TRACKING_URI"] = f"file://{MLRUNS_DIR}"
os.environ["TPU_RUNTIME_METRICS_PORTS"] = "8431,8432,8433,8434"

print(f"Checkpoints will be saved to: {CHECKPOINT_DIR}")
print(f"MLflow runs will be saved to: {MLRUNS_DIR}")

# Verify imports
try:
    import deckgym
    import deckgym_openspiel
    print("deckgym and deckgym_openspiel imported successfully.")
except ImportError as e:
    print(f"Import failed: {e}")

# Run script
!python train.py --batch_size 1024 --max_steps 1000 --save_interval 10 --log_interval 1 --test_interval 10 --checkpoint_dir "{CHECKPOINT_DIR}" --win_reward 1.0 --point_reward 0.1 --damage_reward 0.01 --league_decks "deckgym-core/example_decks/mewtwoex.txt" "deckgym-core/example_decks/blastoiseex.txt" "deckgym-core/example_decks/benchmark-electric.txt" --league_rates 1 1 1 



